In [3]:
#!/usr/bin/env python3
import json
import os
from datetime import datetime, timezone
from pathlib import Path

import numpy as np
import pandas as pd
import gspread
from google.oauth2.service_account import Credentials

try:
    import config
    SPREADSHEET_ID = getattr(config, "SPREADSHEET_ID", None)
    SHEET_TELEMETRY = getattr(config, "SHEET_TELEMETRY", "Sheet1")
    SHEET_OCEAN = getattr(config, "SHEET_OCEAN", "OCEAN_Facets")
    CREDENTIALS_FILE = getattr(config, "CREDENTIALS_FILE", None)
except Exception:
    SPREADSHEET_ID = None
    SHEET_TELEMETRY = "Sheet1"
    SHEET_OCEAN = "OCEAN_Facets"
    CREDENTIALS_FILE = None

def resolve_base_dir() -> Path:
    try:
        return Path(__file__).resolve().parent
    except Exception:
        cwd = Path.cwd()
        if (cwd / "rulebook_v2.json").exists():
            return cwd
        if (cwd / "piplines/scoring/rulebook_v2.json").exists():
            return cwd / "piplines/scoring"
        return cwd

BASE_DIR = resolve_base_dir()
RULEBOOK_PATH = BASE_DIR / "rulebook_v2.json"

def resolve_credentials_file() -> Path:
    candidates = []
    env_path = os.getenv("GOOGLE_APPLICATION_CREDENTIALS")
    if env_path:
        candidates.append(Path(env_path))
    if CREDENTIALS_FILE:
        candidates.append(Path(CREDENTIALS_FILE))
    candidates.append(BASE_DIR / "credentials.json")
    candidates.append(BASE_DIR.parent / "genie-game.json")
    candidates.append(BASE_DIR.parent / "credentials.json")
    candidates.append(Path.cwd() / "credentials.json")
    for p in candidates:
        if p is not None and Path(p).expanduser().exists():
            return Path(p).expanduser()
    raise FileNotFoundError("Service account key not found.")

CREDENTIALS_PATH = resolve_credentials_file()

RAW_MAX = 8.0
MODEL_VERSION = "rulebook_v2"
TIMESTAMP_FMT = "%Y-%m-%dT%H:%M:%SZ"

TRAIT_TO_FACETS = {
    "Openness": ["Imagination", "ArtisticInterests", "Emotionality", "Adventurousness", "Intellect", "Liberalism"],
    "Conscientiousness": ["SelfEfficacy", "Orderliness", "Dutifulness", "AchievementStriving", "SelfDiscipline", "Cautiousness"],
    "Extraversion": ["Friendliness", "Gregariousness", "Assertiveness", "ActivityLevel", "ExcitementSeeking", "Cheerfulness"],
    "Agreeableness": ["Trust", "Morality", "Altruism", "Cooperation", "Modesty", "Sympathy"],
    "Neuroticism": ["Anxiety", "Anger", "Depression", "SelfConsciousness", "Immoderation", "Vulnerability"],
}

MAIN_FACETS = [
    "Anxiety", "Anger", "Depression", "SelfConsciousness", "Immoderation", "Vulnerability",
    "Friendliness", "Gregariousness", "Assertiveness", "ActivityLevel", "ExcitementSeeking", "Cheerfulness",
    "Imagination", "ArtisticInterests", "Emotionality", "Adventurousness", "Intellect", "Liberalism",
    "Trust", "Morality", "Altruism", "Cooperation", "Modesty", "Sympathy",
    "SelfEfficacy", "Orderliness", "Dutifulness", "AchievementStriving", "SelfDiscipline", "Cautiousness",
]

REAL_COLS = [
    "Trait_Openness_Real", "Trait_Conscientiousness_Real", "Trait_Extraversion_Real", "Trait_Agreeableness_Real", "Trait_Neuroticism_Real",
    "Facet_Anxiety_Real", "Facet_Anger_Real", "Facet_Depression_Real", "Facet_SelfConsciousness_Real", "Facet_Immoderation_Real", "Facet_Vulnerability_Real",
    "Facet_Friendliness_Real", "Facet_Gregariousness_Real", "Facet_Assertiveness_Real", "Facet_ActivityLevel_Real", "Facet_ExcitementSeeking_Real", "Facet_Cheerfulness_Real",
    "Facet_Imagination_Real", "Facet_ArtisticInterests_Real", "Facet_Emotionality_Real", "Facet_Adventurousness_Real", "Facet_Intellect_Real", "Facet_Liberalism_Real",
    "Facet_Trust_Real", "Facet_Morality_Real", "Facet_Altruism_Real", "Facet_Cooperation_Real", "Facet_Modesty_Real", "Facet_Sympathy_Real",
    "Facet_SelfEfficacy_Real", "Facet_Orderliness_Real", "Facet_Dutifulness_Real", "Facet_AchievementStriving_Real", "Facet_SelfDiscipline_Real", "Facet_Cautiousness_Real",
]

PRED_COLS = [c.replace("_Real", "_Pred") for c in REAL_COLS]

RULEBOOK = json.loads(RULEBOOK_PATH.read_text())

def normalize_condition(cond: str) -> str:
    if cond is None:
        return ""
    c = cond.replace("&&", "and").replace("||", "or").replace("null", "None")
    if c.strip().lower() == "else":
        return "True"
    return c

def value_satisfies(cond_str: str, x) -> bool:
    if x is None or (isinstance(x, float) and np.isnan(x)):
        return False
    cond = normalize_condition(cond_str)
    try:
        return bool(eval(cond, {"__builtins__": {}}, {"x": x}))
    except Exception:
        return False

def pick_range(ranges_dict: dict, x):
    for _, cfg in ranges_dict.items():
        cond = cfg.get("condition")
        if cond is None or cond == "" or cond.strip().lower() == "else":
            return cfg
        if value_satisfies(cond, x):
            return cfg
    return None

def smart_cast(v):
    if v is None or v == "":
        return None
    if isinstance(v, (int, float)):
        return v
    s = str(v).strip()
    if s.lower() in ("true", "false"):
        return s.lower() == "true"
    try:
        f = float(s)
        if f.is_integer():
            return int(f)
        return f
    except Exception:
        return v

def compute_raw(row):
    facet_raw = {}
    for marker, cfg in RULEBOOK.items():
        if marker not in row:
            continue
        x = row[marker]
        r = pick_range(cfg.get("ranges", {}), x)
        if not r:
            continue
        for facet, delta in r.get("facet_effects", {}).items():
            if facet in MAIN_FACETS:
                facet_raw[facet] = facet_raw.get(facet, 0.0) + float(delta)
    return facet_raw

def calibrate_stable(tele_df, ocean_df, shrinkage=0.10):
    real_map = {(str(r.get("PlayerID", "")).strip(), str(r.get("Big5TestID", "")).strip()): r for _, r in ocean_df.iterrows()}
    xs = {f: [] for f in MAIN_FACETS}; ys = {f: [] for f in MAIN_FACETS}
    for _, row in tele_df.iterrows():
        pid = str(row.get("PlayerID", "")).strip()
        bid = str(row.get("Big5TestID", "")).strip()
        real_row = real_map.get((pid, bid))
        if real_row is None:
            continue
        facet_raw = compute_raw(row)
        for f in MAIN_FACETS:
            real_col = f"Facet_{f}_Real"
            if real_col in real_row and pd.notna(real_row[real_col]):
                xs[f].append(facet_raw.get(f, 0.0))
                ys[f].append(float(real_row[real_col]))
    calib = {}
    for f in MAIN_FACETS:
        x = np.array(xs[f]); y = np.array(ys[f])
        if len(x) >= 2 and (x*x).sum() > 0:
            a_ols = (x*y).sum() / (x*x).sum()
            b_ols = y.mean() - a_ols * x.mean()
        else:
            a_ols, b_ols = 10.0/RAW_MAX, 10.0
        a_prior, b_prior = 10.0/RAW_MAX, 10.0
        a = shrinkage * a_ols + (1 - shrinkage) * a_prior
        b = shrinkage * b_ols + (1 - shrinkage) * b_prior
        a = max(0.1, min(a, 3.0))
        calib[f] = (a, b)
    return calib

def score_row_with_calib(row, calib):
    facet_raw = compute_raw(row)
    facet_pred = {}
    for facet in MAIN_FACETS:
        raw = facet_raw.get(facet, 0.0)
        a, b = calib.get(facet, (10.0 / RAW_MAX, 10.0))
        score = a * raw + b
        score = max(0.0, min(20.0, score))
        facet_pred[facet] = int(round(score))
    trait_pred = {}
    for trait, facets in TRAIT_TO_FACETS.items():
        raw_sum = sum(facet_pred.get(f, 0.0) for f in facets)
        trait_pred[trait] = int(round(max(0.0, min(120.0, raw_sum))))
    return facet_raw, facet_pred, trait_pred

def print_calibration_stats(calib, tele_df):
    coverage = {f: 0 for f in MAIN_FACETS}
    raw_ranges = {f: {"min": float("inf"), "max": -float("inf"), "sum": 0.0, "count": 0} for f in MAIN_FACETS}
    for _, row in tele_df.iterrows():
        facet_raw = compute_raw(row)
        for f, raw in facet_raw.items():
            coverage[f] += 1
            stats = raw_ranges[f]
            stats["min"] = min(stats["min"], raw)
            stats["max"] = max(stats["max"], raw)
            stats["sum"] += raw
            stats["count"] += 1
    print("=== COVERAGE ===")
    for f, cnt in coverage.items():
        print(f"{f}: {cnt}/{len(tele_df)} rows ({cnt/len(tele_df)*100:.1f}%)")
    print("=== RAW STATS ===")
    for f, stats in raw_ranges.items():
        if stats["count"] > 0:
            mean = stats["sum"] / stats["count"]
            print(f"{f}: {stats['min']:.2f} - {mean:.2f} - {stats['max']:.2f}")
    print("=== CALIBRATION ===")
    for f, (a, b) in calib.items():
        print(f"{f}: y = {a:.3f}*x + {b:.3f}")

def compute_errors(ocean_df):
    errors = {}
    for col in REAL_COLS:
        pred_col = col.replace("_Real", "_Pred")
        if pred_col in ocean_df.columns:
            real_vals = pd.to_numeric(ocean_df[col], errors="coerce")
            pred_vals = pd.to_numeric(ocean_df[pred_col], errors="coerce")
            mask = real_vals.notna() & pred_vals.notna()
            if mask.any():
                diff = pred_vals[mask] - real_vals[mask]
                mae = diff.abs().mean()
                rmse = np.sqrt((diff**2).mean())
                errors[pred_col] = {"MAE": mae, "RMSE": rmse, "N": mask.sum()}
    print("=== ERRORS (vs Real) ===")
    for pred_col, m in errors.items():
        print(f"{pred_col}: MAE={m['MAE']:.2f}, RMSE={m['RMSE']:.2f}, N={m['N']}")
    return errors

def sanitize(df: pd.DataFrame) -> pd.DataFrame:
    return df.fillna("")
def get_sheet_df(ws):
    header = [h for h in ws.row_values(1) if str(h).strip()]
    return pd.DataFrame(ws.get_all_records(expected_headers=header))


def main():
    global RULEBOOK
    RULEBOOK = json.loads(RULEBOOK_PATH.read_text())

    if not SPREADSHEET_ID:
        raise SystemExit("SPREADSHEET_ID is not set in config.py")

    creds = Credentials.from_service_account_file(str(CREDENTIALS_PATH), scopes=["https://www.googleapis.com/auth/spreadsheets"])
    gc = gspread.authorize(creds)
    sh = gc.open_by_key(SPREADSHEET_ID)
    ws_tele = sh.worksheet(SHEET_TELEMETRY)
    ws_ocean = sh.worksheet(SHEET_OCEAN)

    tele_df = get_sheet_df(ws_tele)
    ocean_df = get_sheet_df(ws_ocean)

    for col in tele_df.columns:
        tele_df[col] = tele_df[col].map(smart_cast)

    for c in ["PlayerID", "Big5TestID"] + REAL_COLS + PRED_COLS:
        if c not in ocean_df.columns:
            ocean_df[c] = ""

    existing_ids = set(str(x).strip() for x in ocean_df.get("PlayerID", []) if str(x).strip())

    calib = calibrate_stable(tele_df, ocean_df, shrinkage=0.10)
    print_calibration_stats(calib, tele_df)

    ts_str = datetime.now(timezone.utc).strftime(TIMESTAMP_FMT)
    processed = 0

    for _, row in tele_df.iterrows():
        pid = str(row.get("PlayerID", "")).strip()
        if not pid or pid not in existing_ids:
            continue
        bid = str(row.get("Big5TestID", "")).strip()
        facet_raw, facet_pred, trait_pred = score_row_with_calib(row, calib)
        if not facet_pred and not trait_pred:
            continue

        mask = (ocean_df["PlayerID"].astype(str).str.strip() == pid)
        if "Big5TestID" in ocean_df.columns:
            mask = mask & (ocean_df["Big5TestID"].astype(str).str.strip() == bid)
        if not mask.any():
            continue

        oi = ocean_df.index[mask][0]

        for facet, val in facet_pred.items():
            col = f"Facet_{facet}_Pred"
            if col in ocean_df.columns:
                ocean_df.at[oi, col] = val

        for trait, val in trait_pred.items():
            col = f"Trait_{trait}_Pred"
            if col in ocean_df.columns:
                ocean_df.at[oi, col] = val

        if "ModelVersion" in ocean_df.columns:
            ocean_df.at[oi, "ModelVersion"] = MODEL_VERSION
        if "ModelTimestamp" in ocean_df.columns:
            ocean_df.at[oi, "ModelTimestamp"] = ts_str
        if "IsProcessed" in ocean_df.columns:
            ocean_df.at[oi, "IsProcessed"] = "TRUE"

        processed += 1

    cols_order = ["PlayerID", "Big5TestID"] + REAL_COLS + PRED_COLS
    ocean_df = ocean_df[cols_order]

    safe_df = sanitize(ocean_df)
    values = [cols_order] + safe_df.astype(object).values.tolist()
    ws_ocean.update(range_name="A1", values=values)

    compute_errors(ocean_df)
    print(f"Updated {processed} rows into {SHEET_OCEAN} with stable calibration.")

if __name__ == "__main__":
    main()

=== COVERAGE ===
Anxiety: 8/8 rows (100.0%)
Anger: 4/8 rows (50.0%)
Depression: 4/8 rows (50.0%)
SelfConsciousness: 1/8 rows (12.5%)
Immoderation: 5/8 rows (62.5%)
Vulnerability: 5/8 rows (62.5%)
Friendliness: 0/8 rows (0.0%)
Gregariousness: 0/8 rows (0.0%)
Assertiveness: 8/8 rows (100.0%)
ActivityLevel: 8/8 rows (100.0%)
ExcitementSeeking: 8/8 rows (100.0%)
Cheerfulness: 1/8 rows (12.5%)
Imagination: 8/8 rows (100.0%)
ArtisticInterests: 8/8 rows (100.0%)
Emotionality: 8/8 rows (100.0%)
Adventurousness: 2/8 rows (25.0%)
Intellect: 8/8 rows (100.0%)
Liberalism: 1/8 rows (12.5%)
Trust: 0/8 rows (0.0%)
Morality: 0/8 rows (0.0%)
Altruism: 0/8 rows (0.0%)
Cooperation: 0/8 rows (0.0%)
Modesty: 1/8 rows (12.5%)
Sympathy: 0/8 rows (0.0%)
SelfEfficacy: 8/8 rows (100.0%)
Orderliness: 8/8 rows (100.0%)
Dutifulness: 1/8 rows (12.5%)
AchievementStriving: 8/8 rows (100.0%)
SelfDiscipline: 8/8 rows (100.0%)
Cautiousness: 8/8 rows (100.0%)
=== RAW STATS ===
Anxiety: -3.00 - -2.00 - 0.00
Anger: 1.00 - 